In [ ]:
%gui qt
%matplotlib qt
from xmcd_projection import *
import pickle, trimesh
structure_file = 'SOL6-1-3.stl'
magnetisation_file = 'SOL6-1-3_uniform_mag.p'

struct = trimesh.load(structure_file)
magnetisation = np.zeros(struct.vertices.shape)
magnetisation[:, 1] = 1

vis = Visualizer(struct, magnetisation)
vis.show()
vis.save_render('test1.png')

In [241]:
%gui qt
%matplotlib qt
import xmcd_projection
deep_reload(xmcd_projection)
from xmcd_projection import *
structure_file = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\Simulations\spirals\models\spiral_det_in_50w30t.msh"
magnetisation_file = r"F:\PhD_data\Simulations\spiral_automotion_simulations\data\basic_automotion\spiral_det_in_50w30t_005\data\data.30.csv"

# import the structure and magnetisation data
points, faces, tetra = get_mesh_data_from_file(structure_file)
magnetisation, _ = get_magnumfe_magnetisation(magnetisation_file)
# define the trimesh from structure data
struct = define_trimesh(points, faces)

# show the visualiser (look at xmcd_visualisation file for details)
vis = Visualizer(struct, magnetisation)

# define the viewing angles (here all default)
vis.set_camera(azi=30, dist=1e5, center=[0, -700, 0])
# show the rendering
vis.show()

In [46]:
np.array(struct.vertices)

array([[1.54469208e+02, 3.78340442e-14, 3.70000000e+02],
       [1.42346327e+02, 7.25290760e+01, 3.70000000e+02],
       [1.04469208e+02, 2.55875762e-14, 3.70000000e+02],
       ...,
       [1.29884092e+02, 5.98122694e+01, 3.71166624e+02],
       [7.83943821e+01, 1.34693427e+00, 7.97096568e+02],
       [4.23333477e+01, 1.53903998e+00, 7.94528674e+02]])

In [191]:
tetra

array([[2689, 3919, 3079, 3519],
       [1131, 3156, 3125, 3677],
       [1510, 3153, 3118, 3771],
       ...,
       [2788, 4181, 2106, 3564],
       [4181, 2106,  410, 2788],
       [ 410,  411, 2106, 4181]])

In [192]:
faces

array([[3919, 3079, 3519],
       [2689, 3079, 3519],
       [2689, 3919, 3519],
       ...,
       [ 410, 2106, 4181],
       [ 410,  411, 4181],
       [ 410,  411, 2106]])

In [57]:
# ground the structure and get projected points
points[:,2] -= np.min(points[:,2])
struct.vertices = points

struct_projected = project_structure(struct)
projected_points = np.array(struct_projected.vertices)

In [405]:
ax = show_structure_points(struct_projected)
ax = show_structure_points(struct, ax=ax, rescale=False)

In [5]:
projected_points.shape

(4342, 3)

### Pick a point and find a line

In [222]:
np.mean(projected_points[faces[0, :], :], axis=0)

array([ -44.29516754, -650.09517741,    0.        ])

In [223]:
pt = np.mean(projected_points[faces[0, :], :], axis=0)
pt

array([ -44.29516754, -650.09517741,    0.        ])

In [251]:
phi, theta = 90, 15
theta_r = np.deg2rad(theta)
phi_r = np.deg2rad(phi)
p = np.array([np.cos(phi_r) * np.cos(theta_r), np.sin(phi_r)
              * np.cos(theta_r), np.sin(theta_r)])

In [253]:
nx0 = np.cross(p, pt)
y = np.linspace(-1500, 0, 1000)
x = nx0[1]/np.sin(theta_r)*np.ones(y.shape)
z = y*np.tan(theta_r)+nx0[0]

ax.plot(x, y, z, c='red')

In [229]:
ax.scatter(pt[0], pt[1], pt[2], s=100)

In [37]:
ax.set_xlabel('x')

Text(0.04504704806120605, -0.07412763382235021, 'x')

In [381]:
deep_reload(xmcd_projection)
from xmcd_projection import *
edge_faces = get_edge_faces(tetra)

In [385]:
np.array([list(ef) for ef in edge_faces])

array([[ 994,  995,  996],
       [2739,  421,  422],
       [ 560,  553,  542],
       ...,
       [2808,  150,  151],
       [2106, 2788,  410],
       [ 410,  411, 2106]])

### Find the intersecting faces

In [230]:
transf_coord = -np.cross(n, np.cross(n, points - pt))

In [172]:
ax.scatter(transf_coord[:,0], transf_coord[:,1],transf_coord[:,2])

In [173]:
ax.scatter(0,0,0,s=100,c='r')

In [231]:
def triangle_area(pt1, pt2, pt3):
    """Area of a triangle defined by three points. This is for Nx3 points where there are N triangles"""
    return np.linalg.norm(np.cross(pt1-pt3, pt2-pt3, axis=1), axis=1)

def origin_triangle_area(pt1, pt2):
    """Area of a triangle defined by two points and origin"""
    return np.linalg.norm(np.cross(pt1, pt2, axis=1), axis=1)

def origin_in_triangle(triangles):
    """Triangles defined as Nx3x3 array where the first axis is each triange, second axis the points in the triangle and the third axis their coordinates"""
    area_sums = origin_triangle_area(triangles[:,0,:], triangles[:,1,:])+origin_triangle_area(triangles[:,0,:], triangles[:,2,:])+origin_triangle_area(triangles[:,1,:], triangles[:,2,:])
    area_triangles = triangle_area(triangles[:,0, :], triangles[:,1, :],triangles[:,2, :])
    return np.isclose(area_sums, area_triangles)

triangles = np.moveaxis(np.stack((transf_coord[faces[:,0], :], transf_coord[faces[:,1], :], transf_coord[faces[:,2], :])), 0, 1)
triangles_pt = np.moveaxis(np.stack((points[faces[:,0], :], points[faces[:,1], :], points[faces[:,2], :])), 0, 1)

In [232]:
pierced_triangles_indx = origin_in_triangle(triangles)
print(np.any(pierced_triangles_indx))
pierced_triangles = triangles_pt[pierced_triangles_indx, :, :]

True


In [190]:
ax.scatter(pierced_triangles[:, 0, 0], pierced_triangles[:, 0, 1], pierced_triangles[:, 0, 2])
ax.scatter(pierced_triangles[:, 1, 0], pierced_triangles[:, 1, 1], pierced_triangles[:, 1, 2])
ax.scatter(pierced_triangles[:, 2, 0], pierced_triangles[:, 2, 1], pierced_triangles[:, 2, 2])

### Select the intersecting tetrahedra

In [237]:
tetra_pierced = np.nonzero(pierced_triangles_indx)[0]//4
tetra_pierced_unique = np.unique(tetra_pierced)
n_piercings = np.array([np.sum(t == tetra_pierced) for t in tetra_pierced_unique])
tetra_pierced_true = tetra_pierced_unique[n_piercings==2]

In [236]:
tetra_pierced_unique

array([    0,    77,   160,   420,   623,  1984,  3219,  4022,  4052,
        4779,  5215,  5969,  5982,  6118,  6387,  6427,  6444,  6613,
        6863,  7125,  8258,  8490,  8671,  9023,  9648, 10233, 11233,
       12131, 14009], dtype=int32)

In [235]:
tetra_pierced

array([    0,     0,    77,    77,   160,   160,   420,   420,   623,
         623,  1984,  1984,  3219,  3219,  4022,  4022,  4052,  4052,
        4779,  4779,  5215,  5215,  5969,  5969,  5982,  5982,  6118,
        6118,  6387,  6387,  6427,  6427,  6444,  6444,  6613,  6613,
        6863,  6863,  7125,  7125,  8258,  8258,  8490,  8490,  8671,
        8671,  9023,  9023,  9648,  9648, 10233, 10233, 11233, 11233,
       12131, 12131, 14009, 14009], dtype=int32)

In [219]:
pierced_triangles_indx[tetra_pierced_true[0]*4:tetra_pierced_true[0]*4+4]

array([ True,  True,  True,  True])

### Take one of the intersecting tetrahedra and get the length of the line within it

In [363]:
# get the two faces that were pierced
i = 0
tt = tetra_pierced_true[i]
selected_faces = faces[pierced_triangles_indx, :][tetra_pierced==i,:]
selected_triangles = triangles_pt[pierced_triangles_indx, :, :][tetra_pierced==i,:, :]

In [339]:
selected_triangles

array([[[ 6.24447700e+00,  6.41971206e-03, -2.39586916e-02],
        [-4.44955680e+00,  5.17974840e-01, -1.93310842e+00],
        [-1.79492020e+00, -5.24394552e-01,  1.95706711e+00]],

       [[ 7.27210059e-01, -2.34017582e+00,  8.73365506e+00],
        [ 6.24447700e+00,  6.41971206e-03, -2.39586916e-02],
        [-4.44955680e+00,  5.17974840e-01, -1.93310842e+00]]])

In [352]:
triangles[0,:,:]

array([[ 6.244477  ,  0.00641971, -0.02395869],
       [-4.4495568 ,  0.51797484, -1.93310842],
       [-1.7949202 , -0.52439455,  1.95706711]])

In [343]:
def face_ray_intersection(p, x0, triangle):
    n = np.cross(triangle[:,1,:] - triangle[:,0,:], triangle[:,2,:] - triangle[:,0,:], axis=1)
    A = np.einsum('ij,ij->i', n, triangle[:,0,:])
    c = np.cross(p, x0)
    return (np.outer(A, p) - np.cross(n, c))/n.dot(p)[:, np.newaxis]

test_triangle = np.array([[[1, 0, 1], [1, 0, -2], [-1, 0, 1]], [[0, 1, 1], [0, 1, -2], [-1, 0, 1]]])
face_ray_intersection(np.array([0, 1, 0]), pt, test_triangle)

array([[-44.29516754,   0.        ,   0.        ],
       [-44.29516754, -43.29516754,   0.        ]])

In [369]:
intersections = face_ray_intersection(p, pt, selected_triangles)
distance = np.linalg.norm(intersections[0,:] - intersections[1,:])
print(distance)

2.1805346082255603


In [372]:
points_on_faces = face_ray_intersection(p, pt, triangles_pt)

In [375]:
pierced_triangles_indx2 = origin_in_triangle(triangles_pt - points_on_faces[:,np.newaxis,:])

In [376]:
pierced_triangles_indx2

array([ True, False, False, ..., False, False, False])

In [380]:
np.any(np.logical_and(pierced_triangles_indx, ~pierced_triangles_indx2))

False

### Summary how to do it:
- use only the outside faces for projection and for each of those faces, find the centroid
- get the intersection points: face_ray_intersection(p, pt, triangles_pt) where pt are centroids
- displace the triangles by the points on faces and check if origin in triangle: origin_in_triangle(triangles_pt - points_on_faces[:,np.newaxis,:])
- extract the triangles that are pierced by the beam and calculate the distances between their intersections
- for each tetra calulate the average magnetisation
- calculate the attenuation of the beam for each tetra and sum over the attenuations for all pierced tetra

In [333]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *

structure_file = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\Simulations\spirals\models\spiral_det_in_50w30t.msh"
magnetisation_file = r"F:\PhD_data\Simulations\spiral_automotion_simulations\data\basic_automotion\spiral_det_in_50w30t_005\data\data.30.csv"

mesh = get_mesh(structure_file)
magnetisation, _ = get_magnumfe_magnetisation(magnetisation_file)

In [13]:

# get the mesh data
points, faces, tetra = get_mesh_data(mesh)
triangles = get_triangles_from_faces(points, faces)
# project the points by the beam.
projected_points = project_points(points)
# get only the edge faces for projection
edge_faces = get_edge_faces(tetra)
edge_faces = np.array([list(ef) for ef in edge_faces])
# find the centroid of each of the edge faces
projected_edge_triangles = get_triangles_from_faces(
    projected_points, edge_faces)
projected_centroids = np.mean(projected_edge_triangles, axis=1)

In [10]:
pt = projected_centroids[1000, :]
p = get_projection_vector(90, 15)

In [106]:

# get the intersections of the ray with the triangle planes
points_on_planes = triangle_ray_intersection(p, pt, triangles)
# check if the intersection is within the triangle
pierced_triangles_indx = origin_in_triangle(
    triangles - points_on_planes[:, np.newaxis, :])
# reshape to be grouped per tetrahedra
pierced_triangles_indx_resh = pierced_triangles_indx.reshape(-1, 4)
n_piercings_per_tetra = pierced_triangles_indx_resh.sum(axis=1)
pierced_tetra_indx = n_piercings_per_tetra.astype(bool)
assert np.all(n_piercings_per_tetra[pierced_tetra_indx] ==
              2), "One of the rays pierces the vertex! Chances of this happening should be infenitesimal..."
# extract the tetrahedra that are pierced by the beam
pierced_tetra_magnetisation = tetra_magnetisation[pierced_tetra_indx, :]
# get the lenght of the pierce for each tetrahedron
pierced_points = points_on_planes[pierced_triangles_indx,:]
distances = np.linalg.norm(pierced_points[::2] - pierced_points[1::2], axis=1)

In [101]:
np.linalg.norm(pierced_points[::2] - pierced_points[1::2], axis=1).shape

(22,)

In [444]:
import xmcd_projection
deep_reload(xmcd_projection)
from xmcd_projection import *
from xmcd_projection.magnumfe_projection import _get_xmcd_at_point

# xmcd_value = get_xmcd_projection(mesh, magnetisation)

In [335]:
xmcd_projection

array([-14.7744514 , -67.87175865,   5.29302556, ...,  13.86047656,
       -84.80718415, -86.98089109])

In [185]:

# get the mesh data
points, faces, tetra = get_mesh_data(mesh)
# get the tetrahedra magnetisation
tetra_magnetisation = np.mean(
    [magnetisation[tetra[:, i], :] for i in range(4)], axis=0)
# get the beam direction
p = get_projection_vector(90, 15)
# get the triangles defining the mesh
triangles = get_triangles_from_faces(points, faces)

# project the points by the beam.
projected_points = project_points_by_vector(points, p)
# get only the edge faces for projection
edge_faces = get_edge_faces(tetra)
edge_faces = np.array([list(ef) for ef in edge_faces])
# find the centroid of each of the edge faces
projected_edge_triangles = get_triangles_from_faces(
    projected_points, edge_faces)
projected_centroids = np.mean(projected_edge_triangles, axis=1)
# use each of the centroids as the origin of the projection ray,
# and calculate what the xmcd through that centroid is
func = partial(get_pt_ray_piercings,
                   p=p,
                   triangles=triangles)



In [445]:
func = partial(get_pt_ray_piercings,
                   p=p,
                   triangles=triangles)
xmcd_value = np.apply_along_axis(func, 1, projected_centroids[:20,:])

ValueError: could not broadcast input array from shape (2,43) into shape (2,15)

In [447]:
res = func(projected_centroids[0,:])

In [449]:
res.shape

(2, 15)

In [437]:
distances

array([2.15938968, 0.20579818, 0.1799475 , 5.61969506, 0.13621091,
       4.51656476, 0.94370893, 0.8927599 , 1.51242725, 1.75225028,
       2.6890606 , 4.79218631, 0.40163423, 0.80501206, 4.57814454])

In [439]:
res = np.vstack((distances, piercings))

In [441]:
res[0, :]

array([2.15938968, 0.20579818, 0.1799475 , 5.61969506, 0.13621091,
       4.51656476, 0.94370893, 0.8927599 , 1.51242725, 1.75225028,
       2.6890606 , 4.79218631, 0.40163423, 0.80501206, 4.57814454])

In [443]:
res[1, :].astype(int)

array([    6,    70,  3131,  3263,  4553,  6437,  6453,  9712,  9946,
       11568, 11991, 12737, 13164, 15861, 15862])

In [339]:
xmcd_value = xmcd_projection.copy()

In [403]:
import xmcd_projection
deep_reload(xmcd_projection)
from xmcd_projection import *
from xmcd_projection.magnumfe_projection import get_xmcd_at_point

In [404]:
struct = define_trimesh(points, faces)
# ground the structure
struct.vertices[:,2] -= struct.vertices[:,2].min()
# project the points by the beam.
projected_points = project_points(struct.vertices)
projected_struct = define_trimesh(projected_points, edge_faces)
face_magnetisation = np.mean(
        [magnetisation[struct.faces[:, i], :] for i in range(3)], axis=0)
mag_colors = magnetisation_to_color(face_magnetisation)

vis = MagnumfeVisualizer(struct, projected_struct, xmcd_value, struct_colors=mag_colors)
vis.show()

[[0.25882353 0.25882353 0.25882353 1.        ]
 [0.25882353 0.25882353 0.25882353 1.        ]
 [0.25882353 0.25882353 0.25882353 1.        ]
 ...
 [0.25882353 0.25882353 0.25882353 1.        ]
 [0.25882353 0.25882353 0.25882353 1.        ]
 [0.25882353 0.25882353 0.25882353 1.        ]]


In [409]:
ax = show_structure_points(struct)
show_structure_points(projected_struct, ax=ax)

In [189]:
# get the intersections of the ray with the triangle planes
points_on_planes = triangle_ray_intersection(p, pt, triangles)
# check if the intersection is within the triangle
pierced_triangles_indx = origin_in_triangle(
    triangles - points_on_planes[:, np.newaxis, :])
# reshape to be grouped per tetrahedra
pierced_triangles_indx_resh = pierced_triangles_indx.reshape(-1, 4)
n_piercings_per_tetra = pierced_triangles_indx_resh.sum(axis=1)
pierced_tetra_indx = n_piercings_per_tetra.astype(bool)

In [193]:
pierced_triangles_indx_resh.shape

(16050, 4)

In [205]:
selected_tetra = pierced_triangles_indx_resh[n_piercings_per_tetra == 1,:]

In [233]:
np.nonzero(n_piercings_per_tetra == 2)[0]//4

array([ 810, 2488, 3059, 3087, 3092, 3279, 3515, 3576, 3576], dtype=int32)

In [191]:
import mpl_toolkits.mplot3d as m3

In [204]:
tetra

array([[2689, 3919, 3079, 3519],
       [1131, 3156, 3125, 3677],
       [1510, 3153, 3118, 3771],
       ...,
       [2788, 4181, 2106, 3564],
       [4181, 2106,  410, 2788],
       [ 410,  411, 2106, 4181]])

In [321]:
triangle_indx = np.zeros(pierced_triangles_indx_resh.shape).astype(bool)
triangle_indx[n_piercings_per_tetra == 1,:]=True
triangle_indx = triangle_indx.flatten()
selected_triangles = triangles[triangle_indx,:]

In [326]:

from mpl_toolkits.mplot3d import Axes3D

# Create a new plot
fig = plt.figure()
ax = fig.add_subplot((111), projection='3d', proj_type='ortho')

vts = selected_triangles.reshape(-1, 3)
ax.plot_trisurf(vts[:,0], vts[:,1], vts[:,2])
ax.scatter(vts[:,0], vts[:,1], vts[:,2], 'ko')
ax.set_xlabel('x')

Text(0.5, 0, 'x')

In [327]:
phi, theta = 90, 15
theta_r = np.deg2rad(theta)
phi_r = np.deg2rad(phi)
p = np.array([np.cos(phi_r) * np.cos(theta_r), np.sin(phi_r)
              * np.cos(theta_r), np.sin(theta_r)])
nx0 = np.cross(p, pt)
# y = np.linspace(-2600, 100, 100)
y = np.linspace(vts[:,1].min(), vts[:,1].max(), 100)
# y = np.linspace(points_on_planes[:,1].min(), points_on_planes[:,1].max(), 100)
x = nx0[1]/np.sin(theta_r)*np.ones(y.shape)
z = y*np.tan(theta_r)+nx0[0]/np.cos(theta_r)

ax.plot(x, y, z, c='red')

In [284]:
ax.scatter(pt[0],pt[1], pt[2])

In [328]:
points_on_planes = triangle_ray_intersection(p, pt, selected_triangles)


ax.scatter(points_on_planes[:,0], points_on_planes[:,1], points_on_planes[:,2])

In [165]:

# define the trimesh from structure data
struct = define_trimesh(points, faces)

struct_projected = project_structure(struct)
ax = show_structure_points(struct_projected)

In [331]:

# check if the intersection is within the triangle
pierced_triangles_indx = origin_in_triangle(
    selected_triangles - points_on_planes[:, np.newaxis, :])

In [332]:

ax.scatter(points_on_planes[pierced_triangles_indx,0], points_on_planes[pierced_triangles_indx,1], points_on_planes[pierced_triangles_indx,2], s=100, c='r')

In [330]:
pierced_triangles_indx

array([False,  True, False,  True, False,  True,  True, False,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False, False,  True, False, False,  True,  True,  True, False,
       False, False, False,  True,  True,  True, False, False,  True])